# Calculations for the plumes/transport of CO from Biomass Burning manuscript 

## All that is needed should be imported here. The plan is to make all calculations in this notebook

In [3]:
%matplotlib inline
import os
import sys
import glob
import numpy as np
import scipy as sc
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
import matplotlib as mpl
from matplotlib import pyplot as plt
import itertools
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import LineCollection
import cartopy.feature as cfeature
from cartopy.mpl.patch import geos_to_path
from statsmodels.tsa.seasonal import seasonal_decompose

### Basic paths that we will need further down

In [4]:
projpath = '.'
inpath = os.path.join(projpath,'netcdf/')
figspath = os.path.join(projpath,'figs')

### Functions will be added here to make our life easier

In [5]:
#===============================================================================
def dxyp(lats, lons):
#===============================================================================
    """
    Method to be used to convert square meters to grids depending on
    latitudes.
    """
#-------------------------------------------------------------------------------
    from numpy import arange,zeros,pi,sin,cos
#-------------------------------------------------------------------------------
    dxyp = zeros((lats))
    earth_rad = 6370000.
    dlat = pi/lats
    dlon = 2.*pi/lons
    fjeq = 0.5 * (1+lats)
    dd = 2.*(earth_rad**2)*(2.*pi/lons)*sin(0.5*dlat)
    areag = 0.
    for j in arange(1,lats+1):
        dxyp[j-1] = dd*cos(dlat*(j-fjeq))
        areag += dxyp[j-1]
#    for j in arange(lats):
#        dxyp[j] = dd*cos(dlat*(j-fjeq))
#        areag += dxyp[j]
    dxyp[0] *= lons
    dxyp[lats - 1] *= lons
    areag *= lons
#-------------------------------------------------------------------------------
    return dxyp
#-------------------------------------------------------------------------------


In [6]:
def get_reg_data(htap,region,sr,outres=False):
    for code, name in htap[sr].attrs.items():
        if name == region:
            outcode=int(code)
            
    da = htap[sr].where(htap[sr] == outcode, other = np.nan)
    da = da.where(da.isnull(), other = 1)
    if (outres):
        da = da.reindex(lat=outres.lat, lon=outres.lon, method='nearest')
    return da

### Calculate what is needed for the emission table

In [7]:
em = xr.open_dataset(os.path.join(inpath,'emissions_co.nc'))

In [8]:
#### Change units of emissions from kg/m2/s to kg/m2/month since these are monthly data
for d,date in enumerate(em.time):
    em['co'][d] = pd.Timestamp(date.values).day * 24 * 60 * 60 * em['co'][d]

In [9]:
em_an = em.groupby('time.year').sum('time')*1e-9

In [10]:
em_se = em.groupby('time.season').sum('time')*1e-9

#### Now split the emissions per HTAP source region and get numbers (maybe climatological?????)

In [11]:
hhtap = xr.open_dataset(os.path.join(inpath,'HTAP_1x1.nc'))

#### Calculate totals per year in Tg and write to file

In [12]:
with open(os.path.join(projpath,'emisdata.txt'), 'w') as datafile:
    datafile.write('{:>5s}'.format('"year"'))
    datafile.write('{:>20s}'.format('"Southern Africa"'))
    datafile.write('{:>20s}'.format('"South East Asia"'))
    datafile.write('{:>20s}'.format('"South America"'))
    datafile.write('{:>20s}'.format('"Russia"'))
    datafile.write('{:>20s}'.format('"Oceania"'))
    datafile.write('{:>20s}'.format('"Central America"'))
    datafile.write('{:>20s}'.format('"North America"'))
    datafile.write('{:>20s}'.format('"India"'))
    datafile.write('{:>20s}'.format('"Central Asia"'))
    datafile.write('{:>20s}'.format('"China"'))
    datafile.write('{:>20s}'.format('"Europe"'))
    datafile.write('{:>20s}'.format('"North Africa"'))
    datafile.write('{:>20s}'.format('"Saudi Arabia"'))
    datafile.write('{:>20s}'.format('"Oceans"'))
    datafile.write('{:>20s}'.format('"Arctic circle"'))
    datafile.write('{:>20s}'.format('"Antarctic"'))    
    datafile.write('{:>20s}'.format('"total"'))
    datafile.write('\n')
    for y,date in enumerate(em_an.year):
        loc_em = em_an["co"][y]*dxyp(180,360).repeat(360).reshape(180,360)

        datafile.write(f'{date.values:<5d}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Sub Saharan/sub Sahel Africa",                                                "source")).sum().values:15.3f}')          
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "South East Asia",                                                             "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "S. America",                                                                  "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Russia, Belarussia, Ukraine",                                                 "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Pacific, Australia and New Zealand",                                          "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Mexico, Central America, Caribbean, Guyanas, Venezuela, Colombia",            "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "US and Canada (up to 66 N)",                                                  "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "South Asia (India, Nepal, Pakistan,Afghanistan, Bangladesh, Sri Lanka)",      "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Central Asia (Kazakhstan, Uzbekistan, Kyrgyzstan, Tajikistan, Turkmenistan)", "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "East Asia (China, Korea, Japan)",                                             "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Western and Eastern EU and Turkey",                                           "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Northern Africa and Sahara and Sahel",                                        "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Middle East (S. Arabia, Oman, Iran, Iraq, etc)",                              "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Non-arctic/Antarctic Ocean",                                                  "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Arctic Circle (North of 66N) and Greenland",                                  "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Antarctic",                                                                   "source")).sum().values:15.3f}')
        datafile.write(f'{loc_em.sum().values:15.3f}')
        datafile.write('\n')

In [13]:
with open(os.path.join(projpath,'emisdata_season.txt'), 'w') as datafile:
    datafile.write('{:>5s}'.format('"season"'))
    datafile.write('{:>20s}'.format('"Southern Africa"'))
    datafile.write('{:>20s}'.format('"South East Asia"'))
    datafile.write('{:>20s}'.format('"South America"'))
    datafile.write('{:>20s}'.format('"Russia"'))
    datafile.write('{:>20s}'.format('"Oceania"'))
    datafile.write('{:>20s}'.format('"Central America"'))
    datafile.write('{:>20s}'.format('"North America"'))
    datafile.write('{:>20s}'.format('"India"'))
    datafile.write('{:>20s}'.format('"Central Asia"'))
    datafile.write('{:>20s}'.format('"China"'))
    datafile.write('{:>20s}'.format('"Europe"'))
    datafile.write('{:>20s}'.format('"North Africa"'))
    datafile.write('{:>20s}'.format('"Saudi Arabia"'))
    datafile.write('{:>20s}'.format('"Oceans"'))
    datafile.write('{:>20s}'.format('"Arctic circle"'))
    datafile.write('{:>20s}'.format('"Antarctic"'))    
    datafile.write('{:>20s}'.format('"total"'))
    datafile.write('\n')
    for y,date in enumerate(em_se.season):
        loc_em = em_se["co"][y]*dxyp(180,360).repeat(360).reshape(180,360)

        datafile.write(f'{date.values:<4s}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Sub Saharan/sub Sahel Africa",                                                "source")).sum().values:15.3f}')          
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "South East Asia",                                                             "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "S. America",                                                                  "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Russia, Belarussia, Ukraine",                                                 "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Pacific, Australia and New Zealand",                                          "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Mexico, Central America, Caribbean, Guyanas, Venezuela, Colombia",            "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "US and Canada (up to 66 N)",                                                  "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "South Asia (India, Nepal, Pakistan,Afghanistan, Bangladesh, Sri Lanka)",      "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Central Asia (Kazakhstan, Uzbekistan, Kyrgyzstan, Tajikistan, Turkmenistan)", "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "East Asia (China, Korea, Japan)",                                             "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Western and Eastern EU and Turkey",                                           "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Northern Africa and Sahara and Sahel",                                        "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Middle East (S. Arabia, Oman, Iran, Iraq, etc)",                              "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Non-arctic/Antarctic Ocean",                                                  "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Arctic Circle (North of 66N) and Greenland",                                  "source")).sum().values:15.3f}')
        datafile.write(f'{(loc_em*get_reg_data(hhtap, "Antarctic",                                                                   "source")).sum().values:15.3f}')
        datafile.write(f'{loc_em.sum().values:15.3f}')
        datafile.write('\n')

In [ ]:
emdf = pd.read_csv(os.path.join(projpath,'emisdata.txt'), delim_whitespace=True, index_col='year')
emdf['totalsum'] = emdf.sum(axis=1)-emdf['total']
emdf = pd.concat([emdf, emdf.apply(['mean'])])
emdf = pd.concat([emdf, emdf.apply(['median'])])

In [15]:
emdfse = pd.read_csv(os.path.join(projpath,'emisdata_season.txt'), delim_whitespace=True, index_col='season')

In [17]:
emdfse['totalsum'] = emdfse.sum(axis=1)-emdfse['total']

In [18]:
emdfpc = emdf.copy()

In [19]:
for reg in [f for f in emdf.columns if not f.startswith('total')]:
    emdfpc[f'{reg}'] = emdf[reg]/emdf['totalsum']*100
emdfpc = emdfpc.drop('total', axis=1)
emdfpc['totalsum'] = np.nan
emdfpc['totalsum'] = emdfpc.sum(axis=1)

In [20]:
emdfpc['Others']=100 - emdfpc[['Southern Africa', 'South East Asia', 'South America', 'Russia', 'Oceania','Central America', 'North America']].sum(axis=1)

In [32]:
if not os.path.isdir(figspath): os.mkdir(figspath)

In [ ]:
fig = plt.figure(figsize=(6,3), dpi=300)
ax = fig.add_subplot(111)
ax2 = ax.twiny()
emdfpc.drop(['totalsum',
             'India', 
             'Central Asia',
             'China',
             'Europe',
             'North Africa',
             'Saudi Arabia',
             'Oceans',
             'Arctic circle',
             'Antarctic'],axis=1).plot(ax=ax,
                                    kind='bar',
                                    stacked=True, 
                                    colormap='tab10')

ax.axhline(y=90, color='k', linestyle='--', alpha=0.5)
ax.axhline(y=95, color='r', linestyle='--', alpha=0.5)
ax.grid(which='minor', axis='y')
ax.set_xlabel('Year')#, size='large')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)#, size='medium')
ax.set_ylabel('% contribution')#, size='large')
ax.set_yticks(np.arange(0,101,10))
ax.set_yticklabels(np.arange(0,101,10))#, size='medium')
ax.text(-1.35,93,'95', color='r', alpha=0.5, size='small')
ax.set_ylim(0,100)
ax.set_title('Contribution to BB emissions')#, size='large')
ax.legend(loc=(1.01,0.3), frameon=False)

nticks = emdf['total'].shape[0]
tick_locs = np.linspace(1./nticks/2.,1-(1./nticks/2.),nticks)
ax2.set_xticks(tick_locs, ['%.1f' % f for f in emdf['total']], rotation=45)
ax2.set_xlabel('Total annual CO BB emissions (Tg)')

fig.savefig(os.path.join(figspath,'fig1.png'), bbox_inches='tight');

In [23]:
del(em)

### Create a plot of the climatological  emissions

#### reopen to be sure its clean

In [24]:
em = xr.open_dataset(os.path.join(inpath,'emissions_co.nc'))

In [26]:
secofday = 24 *  60 * 60

In [27]:
monthly = em.groupby('time.month').mean('time')

In [28]:
monthly['secs'] = xr.DataArray(np.asarray([31,28,31,30,31,30,31,31,30,31,30,31]) * secofday, coords={'month':monthly.month})
toplot = ((monthly.co * monthly.secs).sum('month')/monthly.secs.sum())

toplot = toplot.where(toplot!=0, other=np.nan)
toplot = toplot.values[::-1,:]

In [ ]:
f = plt.figure(figsize=(8,6),dpi=300)
ax = f.add_subplot(111, projection = ccrs.Robinson())
ax.coastlines()
p = ax.imshow(toplot,  cmap='jet', transform=ccrs.PlateCarree(), norm=mpl.colors.LogNorm(vmin=1e-15, vmax=1e-7))
cbar = plt.colorbar(p, ax=ax, orientation='horizontal', aspect=50, pad=0.05, label=r'CO [kg/m$^2$/s]')
ax.set_title('Climatological CO emissions (1994-2015)')
f.savefig(os.path.join(figspath,'figS6.png'), bbox_inches='tight')
#plt.close(f)

In [30]:
plt.close(f)

In [31]:
del(em)

## Now create the centerpiece of the manuscript

In [33]:
marked = xr.open_dataset(os.path.join(inpath,'marked.nc'))

In [34]:
regions = ['CO_SAf', 
           'CO_CAm', 
           'CO_Chi', 
           'CO_Eur', 
           'CO_Ind', 
           'CO_NAf', 
           'CO_NAm', 
           'CO_Oce', 
           'CO_Rus', 
           'CO_SAm', 
           'CO_SEA', 
           'CO_Sta', 
           'CO_SAr']
rname = ['Southern Africa',
         'Central America',
         'China',
         'Europe',
         'India',
         'North Africa',
         'North America',
         'Oceania',
         'Russia',
         'South America',
         'South East Asia',
         'Central Asia',
         'Saudi Arabia']

rr = np.asarray([regions,rname]).T

In [ ]:
rect = 0.25,0.25,0.5,0.5
rects = {'Central America': [ 0.02, 0.75, 0.19, 0.21],
         'North America'  : [ 0.22, 0.75, 0.19, 0.21],
         'Europe'         : [ 0.42, 0.75, 0.19, 0.21],
         'Central Asia'   : [ 0.62, 0.75, 0.19, 0.21],
         'Russia'         : [ 0.82, 0.75, 0.19, 0.21],
         'North Africa'   : [ 0.02, 0.40, 0.23, 0.21],
         'China'          : [ 0.82, 0.50, 0.19, 0.21],
         'South East Asia': [ 0.82, 0.25, 0.19, 0.21],
         'South America'  : [ 0.02, 0.00, 0.19, 0.21],
         'Southern Africa': [ 0.22, 0.00, 0.19, 0.21],
         'Saudi Arabia'   : [ 0.42, 0.00, 0.19, 0.21],       
         'India'          : [ 0.62, 0.00, 0.19, 0.21],
         'Oceania'        : [ 0.82, 0.00, 0.19, 0.21],}

f = plt.figure(figsize=(18,12),dpi=300)

axs = f.add_axes(rect, projection=ccrs.Robinson())
cmap = plt.get_cmap('jet',rr.shape[0]+1)
norm = mpl.colors.Normalize(vmin=0,vmax=rr.shape[0])
mapargs=dict(ax=axs, 
             cmap='jet',
             levels=rr.shape[0]+1,
             vmin=0, 
             vmax=rr.shape[0], 
             alpha=0.5, 
             add_colorbar=False,
             transform=ccrs.PlateCarree()
            )
contargs=dict(ax=axs,
              alpha=1,
              norm=None,
              transform=ccrs.PlateCarree()
             )
crs = ccrs.PlateCarree()
bins = 5
for i,reg in enumerate(rr):
    dloc = (marked[reg[0]]*1e9).groupby('time.year').mean('time').mean('year').sum('lev')
    rule = dloc.values.mean() + 2 * dloc.values.std()#dloc.values.min() + 2 * dloc.values.std()
    p_ = dloc.plot.contour(**contargs,
                           levels = [rule],
                           colors=mpl.colors.to_hex(mpl.cm.ScalarMappable(norm,cmap).to_rgba(i))
                          )
    dloc = dloc.where(dloc>rule, other=np.nan).where(dloc<rule, other=i+0.5)    
    pa = dloc.plot.imshow(**mapargs)

    dd3d = (marked[reg[0]]*1e9).groupby('time.year').mean('time').mean('year')
    rule = dd3d.mean() + 2. * dd3d.std()
    dd3d = dd3d.where(dd3d>rule, other=np.nan)
    dd3d = dd3d.where(dd3d.lat<82, other=np.nan)
    dd3d = dd3d.transpose('lat','lon','lev')
    ax = f.add_axes(rects[reg[1]], projection='3d')
    ax.set_xlim([-180, 180])
    ax.set_ylim([-90, 90])
    ax.set_zlim(bottom=0, top=20)
    target_projection = ccrs.PlateCarree()
    feature = cfeature.NaturalEarthFeature('physical', 'coastline', '110m')
    geoms = feature.geometries()
    geoms = [target_projection.project_geometry(geom, feature.crs)
             for geom in geoms]
    paths = list(itertools.chain.from_iterable(geos_to_path(geom) for geom in geoms))
    # At this point, we start working around mpl3d's slightly broken interfaces.
    # So we produce a LineCollection rather than a PathCollection.
    segments = []
    for path in paths:
        vertices = [vertex for vertex, _ in path.iter_segments()]
        vertices = np.asarray(vertices)
        segments.append(vertices)
    lc = LineCollection(segments, color='grey')
    ax.add_collection3d(lc)
    ax.set_xlabel('lon')
    ax.set_ylabel('lat')
    ax.set_zlabel('level')

    loc = dd3d.values.flatten()
    x,y,z = np.meshgrid(dd3d.lon,dd3d.lat,dd3d.lev)
    p = ax.scatter(x.flatten(),
                   y.flatten(),
                   z.flatten(),
                   c = loc,
                   s=10,
                   cmap='jet',
                   vmin=0,
                   vmax=25,
#                   vmax=len(rname),
                   alpha=0.2,
                   linewidths=0
                   )

    plt.locator_params(axis='x', nbins=7)
    plt.locator_params(axis='y', nbins=7)
    plt.locator_params(axis='z', nbins=5)
    ax.view_init(30, 240)
    ax.set_title(reg[1])
    if (reg[1] == 'North Africa'):
        cbar = plt.colorbar(p, ax=ax, location='left', aspect=30, pad=0.14, label='Biomass Burning CO (ppb)', shrink=2)
        cbar.set_alpha(0.8)
        cbar.draw_all()
    
cbar = plt.colorbar(pa, ax=axs, ticks=np.arange(0.5,rr.shape[0]+1), orientation='horizontal', aspect=50, pad=0.04)
cbar.ax.set_xticklabels(np.asarray(list(rr.T[1])+['Placeholder']),rotation=45)
axs.coastlines()
axs.set_global()

f.savefig(os.path.join(figspath,'fig2.png'), bbox_inches='tight')

In [ ]:
season='MAM'
rect = 0.25,0.25,0.5,0.5
rects = {'Central America': [ 0.02, 0.75, 0.19, 0.21],
         'North America'  : [ 0.22, 0.75, 0.19, 0.21],
         'Europe'         : [ 0.42, 0.75, 0.19, 0.21],
         'Central Asia'   : [ 0.62, 0.75, 0.19, 0.21],
         'Russia'         : [ 0.82, 0.75, 0.19, 0.21],
         'North Africa'   : [ 0.02, 0.40, 0.23, 0.21],
         'China'          : [ 0.82, 0.50, 0.19, 0.21],
         'South East Asia': [ 0.82, 0.25, 0.19, 0.21],
         'South America'  : [ 0.02, 0.00, 0.19, 0.21],
         'Southern Africa': [ 0.22, 0.00, 0.19, 0.21],
         'Saudi Arabia'   : [ 0.42, 0.00, 0.19, 0.21],       
         'India'          : [ 0.62, 0.00, 0.19, 0.21],
         'Oceania'        : [ 0.82, 0.00, 0.19, 0.21],}

f = plt.figure(figsize=(18,12),dpi=300)

axs = f.add_axes(rect, projection=ccrs.Robinson())
cmap = plt.get_cmap('jet',rr.shape[0]+1)
norm = mpl.colors.Normalize(vmin=0,vmax=rr.shape[0])
mapargs=dict(ax=axs, 
             cmap='jet',
             levels=rr.shape[0]+1,
             vmin=0, 
             vmax=rr.shape[0], 
             alpha=0.5, 
             add_colorbar=False,
             transform=ccrs.PlateCarree()
            )
contargs=dict(ax=axs,
              alpha=1,
              norm=None,
              transform=ccrs.PlateCarree()
             )
crs = ccrs.PlateCarree()
bins = 5
for i,reg in enumerate(rr):
#    dloc = (marked[reg[0]]*1e9).groupby('time.year').mean('time').mean('year').sum('lev')
    dloc = (marked[reg[0]]*1e9).groupby('time.season').mean('time').sum('lev').sel(season=season)
    rule = dloc.values.mean() + 2 * dloc.values.std()
    p_ = dloc.plot.contour(**contargs,
                           levels = [rule],
                           colors=mpl.colors.to_hex(mpl.cm.ScalarMappable(norm,cmap).to_rgba(i))
                          )
    dloc = dloc.where(dloc>rule, other=np.nan).where(dloc<rule, other=i+0.5)    
    pa = dloc.plot.imshow(**mapargs)

#    dd3d = (marked[reg[0]]*1e9).groupby('time.year').mean('time').mean('year')
    dd3d = (marked[reg[0]]*1e9).groupby('time.season').mean('time').sel(season=season)
    rule = dd3d.mean() + 2. *dd3d.std()
    dd3d = dd3d.where(dd3d>rule, other=np.nan)#.where(dd3d<rule, other=r+0.5)
    dd3d = dd3d.where(dd3d.lat<82, other=np.nan)
    dd3d = dd3d.transpose('lat','lon','lev')
    ax = f.add_axes(rects[reg[1]], projection='3d')
    ax.set_xlim([-180, 180])
    ax.set_ylim([-90, 90])
    ax.set_zlim(bottom=0, top=20)
    target_projection = ccrs.PlateCarree()
    feature = cfeature.NaturalEarthFeature('physical', 'coastline', '110m')
    geoms = feature.geometries()
    geoms = [target_projection.project_geometry(geom, feature.crs)
             for geom in geoms]
    paths = list(itertools.chain.from_iterable(geos_to_path(geom) for geom in geoms))
    # At this point, we start working around mpl3d's slightly broken interfaces.
    # So we produce a LineCollection rather than a PathCollection.
    segments = []
    for path in paths:
        vertices = [vertex for vertex, _ in path.iter_segments()]
        vertices = np.asarray(vertices)
        segments.append(vertices)
    lc = LineCollection(segments, color='grey')
    ax.add_collection3d(lc)
    ax.set_xlabel('lon')
    ax.set_ylabel('lat')
    ax.set_zlabel('level')

    loc = dd3d.values.flatten()
    x,y,z = np.meshgrid(dd3d.lon,dd3d.lat,dd3d.lev)
    p = ax.scatter(x.flatten(),
                   y.flatten(),
                   z.flatten(),
                   c = loc,
                   s=10,
                   cmap='jet',
                   vmin=0,
                   vmax=25,
#                   vmax=len(rname),
                   alpha=0.2,
                   linewidths=0
                   )

    plt.locator_params(axis='x', nbins=7)
    plt.locator_params(axis='y', nbins=7)
    plt.locator_params(axis='z', nbins=5)
    ax.view_init(30, 240)
    ax.set_title(reg[1])
    if (reg[1] == 'North Africa'):
        cbar = plt.colorbar(p, ax=ax, location='left', aspect=30, pad=0.14, label='Biomass Burning CO (ppb)', shrink=2)
        cbar.set_alpha(0.8)
        cbar.draw_all()
    
cbar = plt.colorbar(pa, ax=axs, ticks=np.arange(0.5,rr.shape[0]+1), orientation='horizontal', aspect=50, pad=0.04)
cbar.ax.set_xticklabels(np.asarray(list(rr.T[1])+['Placeholder']),rotation=45)
axs.coastlines()
axs.set_global()

f.savefig(os.path.join(figspath,'figS3.png'), bbox_inches='tight')

In [ ]:
season='JJA'
rect = 0.25,0.25,0.5,0.5
rects = {'Central America': [ 0.02, 0.75, 0.19, 0.21],
         'North America'  : [ 0.22, 0.75, 0.19, 0.21],
         'Europe'         : [ 0.42, 0.75, 0.19, 0.21],
         'Central Asia'   : [ 0.62, 0.75, 0.19, 0.21],
         'Russia'         : [ 0.82, 0.75, 0.19, 0.21],
         'North Africa'   : [ 0.02, 0.40, 0.23, 0.21],
         'China'          : [ 0.82, 0.50, 0.19, 0.21],
         'South East Asia': [ 0.82, 0.25, 0.19, 0.21],
         'South America'  : [ 0.02, 0.00, 0.19, 0.21],
         'Southern Africa': [ 0.22, 0.00, 0.19, 0.21],
         'Saudi Arabia'   : [ 0.42, 0.00, 0.19, 0.21],       
         'India'          : [ 0.62, 0.00, 0.19, 0.21],
         'Oceania'        : [ 0.82, 0.00, 0.19, 0.21],}

f = plt.figure(figsize=(18,12),dpi=300)

axs = f.add_axes(rect, projection=ccrs.Robinson())
cmap = plt.get_cmap('jet',rr.shape[0]+1)
norm = mpl.colors.Normalize(vmin=0,vmax=rr.shape[0])
mapargs=dict(ax=axs, 
             cmap='jet',
             levels=rr.shape[0]+1,
             vmin=0, 
             vmax=rr.shape[0], 
             alpha=0.5, 
             add_colorbar=False,
             transform=ccrs.PlateCarree()
            )
contargs=dict(ax=axs,
              alpha=1,
              norm=None,
              transform=ccrs.PlateCarree()
             )
crs = ccrs.PlateCarree()
bins = 5
for i,reg in enumerate(rr):
#    dloc = (marked[reg[0]]*1e9).groupby('time.year').mean('time').mean('year').sum('lev')
    dloc = (marked[reg[0]]*1e9).groupby('time.season').mean('time').sum('lev').sel(season=season)
    rule = dloc.values.mean() + 2 * dloc.values.std()
    p_ = dloc.plot.contour(**contargs,
                           levels = [rule],
                           colors=mpl.colors.to_hex(mpl.cm.ScalarMappable(norm,cmap).to_rgba(i))
                          )
    dloc = dloc.where(dloc>rule, other=np.nan).where(dloc<rule, other=i+0.5)    
    pa = dloc.plot.imshow(**mapargs)

#    dd3d = (marked[reg[0]]*1e9).groupby('time.year').mean('time').mean('year')
    dd3d = (marked[reg[0]]*1e9).groupby('time.season').mean('time').sel(season=season)
    rule = dd3d.mean() + 2. *dd3d.std()
    dd3d = dd3d.where(dd3d>rule, other=np.nan)#.where(dd3d<rule, other=r+0.5)
    dd3d = dd3d.where(dd3d.lat<82, other=np.nan)
    dd3d = dd3d.transpose('lat','lon','lev')
    ax = f.add_axes(rects[reg[1]], projection='3d')
    ax.set_xlim([-180, 180])
    ax.set_ylim([-90, 90])
    ax.set_zlim(bottom=0, top=20)
    target_projection = ccrs.PlateCarree()
    feature = cfeature.NaturalEarthFeature('physical', 'coastline', '110m')
    geoms = feature.geometries()
    geoms = [target_projection.project_geometry(geom, feature.crs)
             for geom in geoms]
    paths = list(itertools.chain.from_iterable(geos_to_path(geom) for geom in geoms))
    # At this point, we start working around mpl3d's slightly broken interfaces.
    # So we produce a LineCollection rather than a PathCollection.
    segments = []
    for path in paths:
        vertices = [vertex for vertex, _ in path.iter_segments()]
        vertices = np.asarray(vertices)
        segments.append(vertices)
    lc = LineCollection(segments, color='grey')
    ax.add_collection3d(lc)
    ax.set_xlabel('lon')
    ax.set_ylabel('lat')
    ax.set_zlabel('level')

    loc = dd3d.values.flatten()
    x,y,z = np.meshgrid(dd3d.lon,dd3d.lat,dd3d.lev)
    p = ax.scatter(x.flatten(),
                   y.flatten(),
                   z.flatten(),
                   c = loc,
                   s=10,
                   cmap='jet',
                   vmin=0,
                   vmax=25,
#                   vmax=len(rname),
                   alpha=0.2,
                   linewidths=0
                   )

    plt.locator_params(axis='x', nbins=7)
    plt.locator_params(axis='y', nbins=7)
    plt.locator_params(axis='z', nbins=5)
    ax.view_init(30, 240)
    ax.set_title(reg[1])
    if (reg[1] == 'North Africa'):
        cbar = plt.colorbar(p, ax=ax, location='left', aspect=30, pad=0.14, label='Biomass Burning CO (ppb)', shrink=2)
        cbar.set_alpha(0.8)
        cbar.draw_all()
    
cbar = plt.colorbar(pa, ax=axs, ticks=np.arange(0.5,rr.shape[0]+1), orientation='horizontal', aspect=50, pad=0.04)
cbar.ax.set_xticklabels(np.asarray(list(rr.T[1])+['Placeholder']),rotation=45)
axs.coastlines()
axs.set_global()

f.savefig(os.path.join(figspath,'figS4.png'), bbox_inches='tight')

In [ ]:
season='SON'
rect = 0.25,0.25,0.5,0.5
rects = {'Central America': [ 0.02, 0.75, 0.19, 0.21],
         'North America'  : [ 0.22, 0.75, 0.19, 0.21],
         'Europe'         : [ 0.42, 0.75, 0.19, 0.21],
         'Central Asia'   : [ 0.62, 0.75, 0.19, 0.21],
         'Russia'         : [ 0.82, 0.75, 0.19, 0.21],
         'North Africa'   : [ 0.02, 0.40, 0.23, 0.21],
         'China'          : [ 0.82, 0.50, 0.19, 0.21],
         'South East Asia': [ 0.82, 0.25, 0.19, 0.21],
         'South America'  : [ 0.02, 0.00, 0.19, 0.21],
         'Southern Africa': [ 0.22, 0.00, 0.19, 0.21],
         'Saudi Arabia'   : [ 0.42, 0.00, 0.19, 0.21],       
         'India'          : [ 0.62, 0.00, 0.19, 0.21],
         'Oceania'        : [ 0.82, 0.00, 0.19, 0.21],}

f = plt.figure(figsize=(18,12),dpi=300)

axs = f.add_axes(rect, projection=ccrs.Robinson())
cmap = plt.get_cmap('jet',rr.shape[0]+1)
norm = mpl.colors.Normalize(vmin=0,vmax=rr.shape[0])
mapargs=dict(ax=axs, 
             cmap='jet',
             levels=rr.shape[0]+1,
             vmin=0, 
             vmax=rr.shape[0], 
             alpha=0.5, 
             add_colorbar=False,
             transform=ccrs.PlateCarree()
            )
contargs=dict(ax=axs,
              alpha=1,
              norm=None,
              transform=ccrs.PlateCarree()
             )
crs = ccrs.PlateCarree()
bins = 5
for i,reg in enumerate(rr):
#    dloc = (marked[reg[0]]*1e9).groupby('time.year').mean('time').mean('year').sum('lev')
    dloc = (marked[reg[0]]*1e9).groupby('time.season').mean('time').sum('lev').sel(season=season)
    rule = dloc.values.mean() + 2 * dloc.values.std()
    p_ = dloc.plot.contour(**contargs,
                           levels = [rule],
                           colors=mpl.colors.to_hex(mpl.cm.ScalarMappable(norm,cmap).to_rgba(i))
                          )
    dloc = dloc.where(dloc>rule, other=np.nan).where(dloc<rule, other=i+0.5)    
    pa = dloc.plot.imshow(**mapargs)

#    dd3d = (marked[reg[0]]*1e9).groupby('time.year').mean('time').mean('year')
    dd3d = (marked[reg[0]]*1e9).groupby('time.season').mean('time').sel(season=season)
    rule = dd3d.mean() + 2. *dd3d.std()
    dd3d = dd3d.where(dd3d>rule, other=np.nan)#.where(dd3d<rule, other=r+0.5)
    dd3d = dd3d.where(dd3d.lat<82, other=np.nan)
    dd3d = dd3d.transpose('lat','lon','lev')
    ax = f.add_axes(rects[reg[1]], projection='3d')
    ax.set_xlim([-180, 180])
    ax.set_ylim([-90, 90])
    ax.set_zlim(bottom=0, top=20)
    target_projection = ccrs.PlateCarree()
    feature = cfeature.NaturalEarthFeature('physical', 'coastline', '110m')
    geoms = feature.geometries()
    geoms = [target_projection.project_geometry(geom, feature.crs)
             for geom in geoms]
    paths = list(itertools.chain.from_iterable(geos_to_path(geom) for geom in geoms))
    # At this point, we start working around mpl3d's slightly broken interfaces.
    # So we produce a LineCollection rather than a PathCollection.
    segments = []
    for path in paths:
        vertices = [vertex for vertex, _ in path.iter_segments()]
        vertices = np.asarray(vertices)
        segments.append(vertices)
    lc = LineCollection(segments, color='grey')
    ax.add_collection3d(lc)
    ax.set_xlabel('lon')
    ax.set_ylabel('lat')
    ax.set_zlabel('level')

    loc = dd3d.values.flatten()
    x,y,z = np.meshgrid(dd3d.lon,dd3d.lat,dd3d.lev)
    p = ax.scatter(x.flatten(),
                   y.flatten(),
                   z.flatten(),
                   c = loc,
                   s=10,
                   cmap='jet',
                   vmin=0,
                   vmax=25,
#                   vmax=len(rname),
                   alpha=0.2,
                   linewidths=0
                   )

    plt.locator_params(axis='x', nbins=7)
    plt.locator_params(axis='y', nbins=7)
    plt.locator_params(axis='z', nbins=5)
    ax.view_init(30, 240)
    ax.set_title(reg[1])
    if (reg[1] == 'North Africa'):
        cbar = plt.colorbar(p, ax=ax, location='left', aspect=30, pad=0.14, label='Biomass Burning CO (ppb)', shrink=2)
        cbar.set_alpha(0.8)
        cbar.draw_all()
    
cbar = plt.colorbar(pa, ax=axs, ticks=np.arange(0.5,rr.shape[0]+1), orientation='horizontal', aspect=50, pad=0.04)
cbar.ax.set_xticklabels(np.asarray(list(rr.T[1])+['Placeholder']),rotation=45)
axs.coastlines()
axs.set_global()

f.savefig(os.path.join(figspath,'figS5.png'), bbox_inches='tight')

In [ ]:
season='DJF'
rect = 0.25,0.25,0.5,0.5
rects = {'Central America': [ 0.02, 0.75, 0.19, 0.21],
         'North America'  : [ 0.22, 0.75, 0.19, 0.21],
         'Europe'         : [ 0.42, 0.75, 0.19, 0.21],
         'Central Asia'   : [ 0.62, 0.75, 0.19, 0.21],
         'Russia'         : [ 0.82, 0.75, 0.19, 0.21],
         'North Africa'   : [ 0.02, 0.40, 0.23, 0.21],
         'China'          : [ 0.82, 0.50, 0.19, 0.21],
         'South East Asia': [ 0.82, 0.25, 0.19, 0.21],
         'South America'  : [ 0.02, 0.00, 0.19, 0.21],
         'Southern Africa': [ 0.22, 0.00, 0.19, 0.21],
         'Saudi Arabia'   : [ 0.42, 0.00, 0.19, 0.21],       
         'India'          : [ 0.62, 0.00, 0.19, 0.21],
         'Oceania'        : [ 0.82, 0.00, 0.19, 0.21],}

f = plt.figure(figsize=(18,12),dpi=300)

axs = f.add_axes(rect, projection=ccrs.Robinson())
cmap = plt.get_cmap('jet',rr.shape[0]+1)
norm = mpl.colors.Normalize(vmin=0,vmax=rr.shape[0])
mapargs=dict(ax=axs, 
             cmap='jet',
             levels=rr.shape[0]+1,
             vmin=0, 
             vmax=rr.shape[0], 
             alpha=0.5, 
             add_colorbar=False,
             transform=ccrs.PlateCarree()
            )
contargs=dict(ax=axs,
              alpha=1,
              norm=None,
              transform=ccrs.PlateCarree()
             )
crs = ccrs.PlateCarree()
bins = 5
for i,reg in enumerate(rr):
#    dloc = (marked[reg[0]]*1e9).groupby('time.year').mean('time').mean('year').sum('lev')
    dloc = (marked[reg[0]]*1e9).groupby('time.season').mean('time').sum('lev').sel(season=season)
    rule = dloc.values.mean() + 2 * dloc.values.std()
    p_ = dloc.plot.contour(**contargs,
                           levels = [rule],
                           colors=mpl.colors.to_hex(mpl.cm.ScalarMappable(norm,cmap).to_rgba(i))
                          )
    dloc = dloc.where(dloc>rule, other=np.nan).where(dloc<rule, other=i+0.5)    
    pa = dloc.plot.imshow(**mapargs)

#    dd3d = (marked[reg[0]]*1e9).groupby('time.year').mean('time').mean('year')
    dd3d = (marked[reg[0]]*1e9).groupby('time.season').mean('time').sel(season=season)
    rule = dd3d.mean() + 2. *dd3d.std()
    dd3d = dd3d.where(dd3d>rule, other=np.nan)#.where(dd3d<rule, other=r+0.5)
    dd3d = dd3d.where(dd3d.lat<82, other=np.nan)
    dd3d = dd3d.transpose('lat','lon','lev')
    ax = f.add_axes(rects[reg[1]], projection='3d')
    ax.set_xlim([-180, 180])
    ax.set_ylim([-90, 90])
    ax.set_zlim(bottom=0, top=20)
    target_projection = ccrs.PlateCarree()
    feature = cfeature.NaturalEarthFeature('physical', 'coastline', '110m')
    geoms = feature.geometries()
    geoms = [target_projection.project_geometry(geom, feature.crs)
             for geom in geoms]
    paths = list(itertools.chain.from_iterable(geos_to_path(geom) for geom in geoms))
    # At this point, we start working around mpl3d's slightly broken interfaces.
    # So we produce a LineCollection rather than a PathCollection.
    segments = []
    for path in paths:
        vertices = [vertex for vertex, _ in path.iter_segments()]
        vertices = np.asarray(vertices)
        segments.append(vertices)
    lc = LineCollection(segments, color='grey')
    ax.add_collection3d(lc)
    ax.set_xlabel('lon')
    ax.set_ylabel('lat')
    ax.set_zlabel('level')

    loc = dd3d.values.flatten()
    x,y,z = np.meshgrid(dd3d.lon,dd3d.lat,dd3d.lev)
    p = ax.scatter(x.flatten(),
                   y.flatten(),
                   z.flatten(),
                   c = loc,
                   s=10,
                   cmap='jet',
                   vmin=0,
                   vmax=25,
#                   vmax=len(rname),
                   alpha=0.2,
                   linewidths=0
                   )

    plt.locator_params(axis='x', nbins=7)
    plt.locator_params(axis='y', nbins=7)
    plt.locator_params(axis='z', nbins=5)
    ax.view_init(30, 240)
    ax.set_title(reg[1])
    if (reg[1] == 'North Africa'):
        cbar = plt.colorbar(p, ax=ax, location='left', aspect=30, pad=0.14, label='Biomass Burning CO (ppb)', shrink=2)
        cbar.set_alpha(0.8)
        cbar.draw_all()
    
cbar = plt.colorbar(pa, ax=axs, ticks=np.arange(0.5,rr.shape[0]+1), orientation='horizontal', aspect=50, pad=0.04)
cbar.ax.set_xticklabels(np.asarray(list(rr.T[1])+['Placeholder']),rotation=45)
axs.coastlines()
axs.set_global()

f.savefig(os.path.join(figspath,'figS2.png'), bbox_inches='tight')

In [ ]:
f = plt.figure(figsize=(18,12),dpi=300)

axs = f.add_subplot(111, projection=ccrs.Robinson())
cmap = plt.get_cmap('jet',rr.shape[0]+1)
norm = mpl.colors.Normalize(vmin=0,vmax=rr.shape[0])
mapargs=dict(ax=axs, 
             cmap='jet',
             levels=rr.shape[0]+1,
             vmin=0, 
             vmax=rr.shape[0], 
             alpha=0.5, 
             add_colorbar=False,
             transform=ccrs.PlateCarree()
            )
contargs=dict(ax=axs,
              alpha=1,
              norm=None,
              transform=ccrs.PlateCarree()
             )
crs = ccrs.PlateCarree()
for i,reg in enumerate(rr):
    dloc = (marked[reg[0]]*1e9).groupby('time.year').mean('time').mean('year').sum('lev')
    rule = 100#dloc.values.mean() + 2*dloc.values.std()#dloc.values.min()+2 * dloc.values.std()
    p_ = dloc.plot.contour(**contargs,
                           levels = [rule],
                           colors=mpl.colors.to_hex(mpl.cm.ScalarMappable(norm,cmap).to_rgba(i))
                          )
    dloc = dloc.where(dloc>rule, other=np.nan).where(dloc<rule, other=i+0.5)
    pa = dloc.plot.imshow(**mapargs)
    
cbar = plt.colorbar(pa, ax=axs, ticks=np.arange(0.5,rr.shape[0]+1), orientation='horizontal', aspect=50, pad=0.04)
cbar.ax.set_xticklabels(np.asarray(list(rr.T[1])+['Placeholder']),rotation=45)
axs.coastlines()
axs.set_global()
axs.add_feature(cfeature.BORDERS)
f.savefig(os.path.join(figspath,'figS1.png'), bbox_inches='tight')
#plt.show()
# plt.close(f)


In [42]:
trp = xr.open_dataset(os.path.join(inpath,'tropospheric_co.nc'))

In [48]:
receptors = xr.open_dataset(os.path.join(inpath,'HTAP_1x1.nc'))

In [44]:
nat = get_reg_data(receptors,'North Atlantic',         'receptor', outres=trp).rename('North Atlantic')
sat = get_reg_data(receptors,'South Atlantic',         'receptor', outres=trp).rename('South Atlantic')
npa = get_reg_data(receptors,'North Pacific',          'receptor', outres=trp).rename('North Pacific')
spa = get_reg_data(receptors,'South Pacific',          'receptor', outres=trp).rename('South Pacific')
ind = get_reg_data(receptors,'Indian Ocean',           'receptor', outres=trp).rename('Indian Ocean')
ant = get_reg_data(receptors,'Antarctic Circle (sea)', 'receptor', outres=trp).rename('Antarctic Circle')
others = (get_reg_data(receptors,'Baltic Sea', 'receptor', outres=trp).fillna(0)+
          get_reg_data(receptors,'Hudson Bay', 'receptor', outres=trp).fillna(0)+
          get_reg_data(receptors,'Mediterranean','receptor', outres=trp).fillna(0)+
          get_reg_data(receptors,'Black and Caspian Sea','receptor', outres=trp).fillna(0))
others = others.where(others>0).rename('Others')

In [45]:
area = xr.DataArray(dxyp(90,120).repeat(120).reshape(90,120), 
                    coords={'lat':nat.lat,'lon':nat.lon})*1e4 #cm2

In [46]:
index = ['Southern Africa',
         'South East Asia',
         'South America',
         'Russia',
         'Oceania',
         'Central America',
         'North America',
         'India',
         'Central Asia',
         'China',
         'Europe',
         'North Africa',
         'Saudi Arabia',
         'total']
regs = ['CO_SAf_tc',
        'CO_SEA_tc',
        'CO_SAm_tc',
        'CO_Rus_tc',
        'CO_Oce_tc',
        'CO_CAm_tc',
        'CO_NAm_tc',
        'CO_Ind_tc',
        'CO_Sta_tc',
        'CO_Chi_tc',
        'CO_Eur_tc',
        'CO_NAf_tc',
        'CO_SAr_tc',
        'CO_tc']
ocean_impact = pd.DataFrame()
for oce in [nat,sat,npa,spa,ind,ant,others]:
    peroce = dict()
    for reg in regs:#[f for f in trp.variables if f.endswith('_tc')]:
        val = (trp[reg].groupby('time.year').sum('time').mean('year')*area*oce).sum()*1e-23*1e-12*28
        peroce[reg] = val.values
    ocean_impact[oce.name] = pd.DataFrame(peroce.values(),index=index)
#        print(reg,val.values)